<a href="https://colab.research.google.com/github/erdenebayrd/Competitive-Coding/blob/master/week8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from sklearn.metrics import accuracy_score

In [ ]:
char_arr = []
for i in range(26):
  char_arr.append(chr(i + ord('a')))

In [ ]:
char_arr

['a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [ ]:
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

seq_data = ["word", "wood", "deep", "dive", "cold", "cool", "load", "love", "kiss", "kind"]

In [ ]:
def make_batch(seq_data):
  input_batch = []
  target_batch = []

  for seq in seq_data:
    input = [num_dic[n] for n in seq[:-1]]
    target = num_dic[seq[-1]]
    input_batch.append(np.eye(dic_len)[input])
    target_batch.append([target])
  return input_batch, target_batch

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
n_input = dic_len
n_hidden = 128
n_class = dic_len
learning_rate = 0.1
total_epoch = 20

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.lstm = nn.LSTM(input_size=n_input, hidden_size=n_hidden, num_layers=1, batch_first=True)
    self.linear = nn.Linear(n_hidden, n_class)

  def forward(self, x):
    x, _ = self.lstm(x)
    x = self.linear(x[:, -1, :])
    x = F.softmax(x, dim=1)
    return x

In [ ]:
net = Net().to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [ ]:
input_batch, target_batch = make_batch(seq_data)
input_batch_torch = torch.from_numpy(np.array(input_batch)).float().to(device)
target_batch_torch = torch.from_numpy(np.array(target_batch)).view(-1).long().to(device)

In [ ]:
for epoch in range(total_epoch):
  net.train()
  optimizer.zero_grad()

  output = net(input_batch_torch)
  loss = criterion(output, target_batch_torch)
  loss.backward()
  optimizer.step()

  _, predicted = torch.max(output, 1)
  accuracy = accuracy_score(predicted.cpu().numpy(), target_batch_torch.cpu().numpy())

  print(f"Epoch: {epoch}, Loss: {loss.item()}, Accuracy: {accuracy}")
print("finished traning")

Epoch: 0, Loss: -0.03893197327852249, Accuracy: 0.1
Epoch: 1, Loss: -0.402195543050766, Accuracy: 0.5
Epoch: 2, Loss: -0.49999865889549255, Accuracy: 0.5
Epoch: 3, Loss: -0.5, Accuracy: 0.5
Epoch: 4, Loss: -0.5, Accuracy: 0.5
Epoch: 5, Loss: -0.5, Accuracy: 0.5
Epoch: 6, Loss: -0.5, Accuracy: 0.5
Epoch: 7, Loss: -0.5, Accuracy: 0.5
Epoch: 8, Loss: -0.5, Accuracy: 0.5
Epoch: 9, Loss: -0.5, Accuracy: 0.5
Epoch: 10, Loss: -0.5, Accuracy: 0.5
Epoch: 11, Loss: -0.5, Accuracy: 0.5
Epoch: 12, Loss: -0.5, Accuracy: 0.5
Epoch: 13, Loss: -0.5, Accuracy: 0.5
Epoch: 14, Loss: -0.5, Accuracy: 0.5
Epoch: 15, Loss: -0.5, Accuracy: 0.5
Epoch: 16, Loss: -0.5, Accuracy: 0.5
Epoch: 17, Loss: -0.5, Accuracy: 0.5
Epoch: 18, Loss: -0.5, Accuracy: 0.5
Epoch: 19, Loss: -0.5, Accuracy: 0.5
finished traning


In [ ]:
predicted_words = []
for i in range(len(predicted.cpu().numpy())):
  ind = predicted.cpu().numpy()[i]
  predicted_words.append(seq_data[i][:-1] + char_arr[ind])

print("\n=== Prediction Result ===")
print("Input:", [w[:3] + " " for w in seq_data])
print("Predicted:", predicted_words)
print("Accuracy:", accuracy)


=== Prediction Result ===
Input: ['wor ', 'woo ', 'dee ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
Predicted: ['word', 'wood', 'deed', 'divd', 'cold', 'cood', 'load', 'lovd', 'kisd', 'kind']
Accuracy: 0.5


In [ ]:
new_seq_data = ["sold", "peep", "miss", "told", "cook", "hope", "live", "mind"]
input_batch_new, target_batch_new = make_batch(new_seq_data)
input_batch_torch_new = torch.from_numpy(np.array(input_batch_new)).float().to(device)

In [ ]:
net.eval()
with torch.no_grad():
  new_output = net(input_batch_torch_new)
  _, new_predicted = torch.max(new_output, 1)

print("\n=== New Prediction Result ===")
print("Input:", [w[:3] + " " for w in new_seq_data])


=== New Prediction Result ===
Input: ['sol ', 'pee ', 'mis ', 'tol ', 'coo ', 'hop ', 'liv ', 'min ']
